## Google doc with required deps here:


https://docs.google.com/document/d/1fUM2bguzMHy6NOR1kX_zfABbEWaU9nwBfdWSabp0PDA/edit

## Is Docker installed?

In [ ]:
docker run hello-world

Let's check we have at least 8GB of memory allocated to docker

In [ ]:
docker run -it ubuntu free -h

## Let's download the required docker images

In [ ]:
#docker image for crispor and crispresso
docker pull pinellolab/crispor_crispresso_nat_prot


In [ ]:
#docker image for cas-offinder
docker pull lucapinello/cas-offinder

In [ ]:
#let's check the current folder
pwd

In [ ]:
#let's create a folder that will contain the genome files for CRISPOR and cas-offinder

In [ ]:
mkdir crispor_genomes

In [ ]:
ls

In [ ]:
#download the human genome assembly hg19
docker run -v $PWD/crispor_genomes:/crisporWebsite/genomes pinellolab/crispor_crispresso_nat_prot downloadGenome hg19 /crisporWebsite/genomes


## 1. Crispor Demo

In [ ]:
echo '>BCL11A' > crispor_input.fa
echo 'CCGAGCCTCTTGAAGCCATTCTTACAGATGATGAACCAGACCACGGCCCGTTGGGAGCTCCAGAAGGGGATCATGACCTCCTCACCTGTGGGCAGTGCCAGATGAACTTCCCATTGGGGGACATTCTTATTTTTATCGAGCACAAACGGAAACAATGCAATGGCAGCCTCTGCTTAGAAAAAGCTGTGGATAAGCCACCTTCCCCTTCACCAATCGAGATGAAAAAAGCATCCAATCCCGTGGAGGTTGGCATCCAGGTCACGCCAGAGGATGACGATTGTTTATCAACGTCATCTAGAGGAATTTGCCCCAAACAGGAACACATAGCAG' >> crispor_input.fa

In [ ]:
cat crispor_input.fa

In [ ]:
#design guides
docker  run  \
-v  $PWD/crispor_genomes:/crisporWebsite/genomes \
-v $PWD/:/DATA \
-w /DATA pinellolab/crispor_crispresso_nat_prot \
crispor.py hg19 crispor_input.fa crispor_output.tsv --satMutDir=./


In [ ]:
!ls 

In [ ]:
#simulate filtering, in this case we take the top 10 guides
docker run \
-v $PWD/:/DATA \
-w /DATA pinellolab/crispor_crispresso_nat_prot \
bash -c "head -n 10 BCL11A_satMutOligos.tsv > BCL11A_satMutOligos_filtered.tsv" 


In [ ]:
head BCL11A_satMutOligos_filtered.tsv

In [ ]:
#create filtered files for the experiment and for CRISPResso analysis
docker run -v $PWD:/DATA -w /DATA pinellolab/crispor_crispresso_nat_prot bash -c "join -1 1 -2 1 BCL11A_satMutOligos_filtered.tsv BCL11A_ontargetAmplicons.tsv -o 2.1,2.2,2.3 > CRISPRessoPooled_amplicons.tsv"



In [ ]:
#create filtered files for CRISPResso analysis
docker run -v $PWD:/DATA -w /DATA pinellolab/crispor_crispresso_nat_prot bash -c "join -1 1 -2 1 BCL11A_satMutOligos_filtered.tsv BCL11A_ontargetPrimers.tsv -o 2.1,2.2,2.3,2.4,2.5,2.6,2.7 > BCL11A_ontargetPrimers_filtered.tsv"

In [ ]:
head  BCL11A_ontargetPrimers_filtered.tsv

## 2. CRISPResso Demo

Let's now try to run CRIPSResso on data from the last guide in this table:  


52forw CCACAGGTGAGGAGGTCATG 59.749 TTTCTCCAACCACAGCCGAG 60.250 CCACAGGTGAGGAGGTCATGATCCCCTTCTGGAGCTCCCAACGGGCCGTGGTCTGGTTCATCATCTGTAAGAATGGCTTCAAGAGGCTCGGCTGTGGTTGGAGAAA TGAACCAGACCACGGCCCGT

The amplicon for the experiment was a little bit different since it was not designed with CRISPRO:

AMPLICON:
AATGTCCCCCAATGGGAAGTTCATCTGGCACTGCCCACAGGTGAGGAGGTCATGATCCCCTTCTGGAGCTCCCAACGGGCCGTGGTCTGGTTCATCATCTGTAAGAATGGCTTCAAGAGGCTCGGCTGTGGTT 

GUIDE:
TGAACCAGACCACGGCCCGT


In [ ]:
#let's download the data
wget http://bcb.dfci.harvard.edu/~lpinello/CRISPResso/reads1.fastq.gz
wget http://bcb.dfci.harvard.edu/~lpinello/CRISPResso/reads2.fastq.gz

In [ ]:
#finally we run CRISPResso
docker  run \
-v $PWD:/DATA \
-w /DATA pinellolab/crispor_crispresso_nat_prot  \
CRISPResso -r1 reads1.fastq.gz -r2 reads2.fastq.gz \
-a AATGTCCCCCAATGGGAAGTTCATCTGGCACTGCCCACAGGTGAGGAGGTCATGATCCCCTTCTGGAGCTCCCAACGGGCCGTGGTCTGGTTCATCATCTGTAAGAATGGCTTCAAGAGGCTCGGCTGTGGTT \
-g TGAACCAGACCACGGCCCGT \
-s 20 \
-q 30 \
-n BCL11A_exon2


## 3. CasOffinder Demo

Finally let's compute all the potential off-target of our guide with cas-offinder with up to 6 mismatches

In [ ]:
#let's take a look to the command syntax
docker run lucapinello/cas-offinder cas-offinder --help

In [ ]:
#now we create the required input file

In [ ]:
pwd

In [ ]:
echo '/DATA/crispor_genomes/hg19/hg19.2bit' > cas_offinder_input.txt
echo 'NNNNNNNNNNNNNNNNNNNNNGG' >> cas_offinder_input.txt
echo 'TGAACCAGACCACGGCCCGTNGG 5' >> cas_offinder_input.txt

In [ ]:
cat cas_offinder_input.txt

In [ ]:
#finally we can run cas-offinder 
docker run \
-v $PWD:/DATA \
-w /DATA  \
lucapinello/cas-offinder  \
cas-offinder cas_offinder_input.txt C cas_offinder_output.txt

In [ ]:
head cas_offinder_output.txt